## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_38.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_38.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [4]:
Total_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 100 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   시술_당시_나이               256344 non-null  object 
 1   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 2   배란_자극_여부               256344 non-null  int64  
 3   배란_유도_유형               256344 non-null  object 
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불임원인여부_불명확             256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-n

In [5]:
Total_categorical_columns = [
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [6]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [7]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

In [8]:
# %pip install autogluon

In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 학습 데이터와 테스트 데이터를 합쳐서 DataFrame 생성
train_data = pd.concat([Total_X_train, Total_y_train], axis=1)
test_data = pd.concat([Total_X_test, Total_y_test], axis=1)

# 타겟 컬럼 이름
label = Total_y_train.name

# 하이퍼파라미터 설정
hyperparameters = {
    "GBM": {},  # LightGBM
    "XGB": {},  # XGBoost
    "CAT": {},  # CatBoost
    "NN_TORCH": {},  # PyTorch Neural Network
    "FASTAI": {},  # FastAI Neural Network
    "RF": {},  # Random Forest
    "XT": {}   # Extra Trees
}

# 하이퍼파라미터 튜닝 설정
hyperparameter_tune_kwargs = {
    'num_trials': 100,
    'searcher': 'bayes',  # Bayesian optimization
    'scheduler': 'local'
}

# 모델 저장 경로
# save_path = "new_start8"
save_path = "C:/autogluon_models"



# AutoGluon 초기화 및 모델 학습
predictor = TabularPredictor(label=label, problem_type="binary", eval_metric="roc_auc", path=save_path)
predictor.fit(train_data, presets="best_quality",
              time_limit=60000,
              hyperparameters=hyperparameters,
              hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
              num_bag_folds=10,
              auto_stack=True,
              refit_full=True,
              calibrate=True,
              num_stack_levels=2
              )

# 앙상블 모델 구성 요소 확인
predictor.leaderboard(silent=True)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=10, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 60000s
AutoGluon will save models to "new_start8\"
AutoGluon Version:  0.6.2
Python Version:     3.7.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Train Data Rows:    205075
Train Data Columns: 100
Label Column: 임신_성공_여부
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6425.49 MB
	Train Data (Original)  Memory Usage: 164.06 MB (2.6% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 63 features to boolean dtype as they only conta

  0%|          | 0/100 [00:00<?, ?it/s]

	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's binary_logloss: 0.487536
[1000]	valid_set's binary_logloss: 0.487352
[1000]	valid_set's binary_logloss: 0.486052
[1000]	valid_set's binary_logloss: 0.486435
[1000]	valid_set's binary_logloss: 0.488425
[1000]	valid_set's binary_logloss: 0.48878
[1000]	valid_set's binary_logloss: 0.488294
[1000]	valid_set's binary_logloss: 0.492269
[1000]	valid_set's binary_logloss: 0.486715
[1000]	valid_set's binary_logloss: 0.491597


	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy


...

[1000]	valid_set's binary_logloss: 0.491294


	0.7383	 = Validation score   (roc_auc)
	165.58s	 = Training   runtime
	1.22s	 = Validation runtime
Fitting model: LightGBM_BAG_L3\T50 ... Training model for up to 12440.88s of the 12440.83s of remaining time.
	Fitting 10 child models (S2F1 - S2F10) | Fitting with SequentialLocalFoldFittingStrategy
	0.7382	 = Validation score   (roc_auc)
	69.89s	 = Training   runtime
	0.56s	 = Validation runtime
Fitting model: LightGBM_BAG_L3\T51 ... Training model for up to 12414.46s of the 12414.42s of remaining time.
	Fitting 10 child models (S2F1 - S2F10) | Fitting with SequentialLocalFoldFittingStrategy
	0.7388	 = Validation score   (roc_auc)
	27.25s	 = Training   runtime
	0.39s	 = Validation runtime
Fitting model: LightGBM_BAG_L3\T52 ... Training model for up to 12403.17s of the 12403.12s of remaining time.
	Fitting 10 child models (S2F1 - S2F10) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's binary_logloss: 0.4913
[1000]	valid_set's binary_logloss: 0.486286
[1000]	valid_set's binary_logloss: 0.491155
[1000]	valid_set's binary_logloss: 0.486557
[1000]	valid_set's binary_logloss: 0.491619
[1000]	valid_set's binary_logloss: 0.48752
[1000]	valid_set's binary_logloss: 0.48871
[1000]	valid_set's binary_logloss: 0.490562
[1000]	valid_set's binary_logloss: 0.487902
[1000]	valid_set's binary_logloss: 0.488747


	0.7383	 = Validation score   (roc_auc)
	205.74s	 = Training   runtime
	1.83s	 = Validation runtime
Fitting model: LightGBM_BAG_L3\T53 ... Training model for up to 12326.47s of the 12326.43s of remaining time.
	Fitting 10 child models (S2F1 - S2F10) | Fitting with SequentialLocalFoldFittingStrategy
	0.7384	 = Validation score   (roc_auc)
	126.3s	 = Training   runtime
	0.99s	 = Validation runtime
Fitting model: LightGBM_BAG_L3\T54 ... Training model for up to 12280.87s of the 12280.82s of remaining time.
	Fitting 10 child models (S2F1 - S2F10) | Fitting with SequentialLocalFoldFittingStrategy
	0.7381	 = Validation score   (roc_auc)
	36.09s	 = Training   runtime
	0.34s	 = Validation runtime
Fitting model: CatBoost_BAG_L3\T1 ... Training model for up to 12266.31s of the 12266.27s of remaining time.
	Fitting 10 child models (S2F1 - S2F10) | Fitting with SequentialLocalFoldFittingStrategy
	0.7389	 = Validation score   (roc_auc)
	207.64s	 = Training   runtime
	0.17s	 = Validation runtime
Fit

XGBoostError: [13:48:39] C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/common/io.cc:102: Opening c:\Users\juneh\OneDrive\바탕 화면\git_Aimers6th\LG_Aimers_6th\code\new_start8\models\XGBoost_BAG_L1\T1\S1F1\xgb.ubj failed: Illegal byte sequence

In [ ]:
# Total_test 데이터의 타겟 변수 예측
Total_pred_proba = predictor.predict_proba(Total_test.drop('ID', axis=1))
Total_test['probability'] = Total_pred_proba.iloc[:, 1]

# 최종 제출 파일 생성
submission = Total_test[['ID', 'probability']]
submission = submission.sort_values(by='ID')

# 제출 파일 저장
submission.to_csv('../submission/code46_all_autogluon.csv', index=False, encoding='utf-8')

데이콘 PUBLIC : 0.7387156458

----

.